# Spark Basics 1

This notebook inroduces two fundamental objects in Spark:
* The Spark Context

* The Resilient Distributed DataSet or RDD

The following code imports all the tools and exercises needed to complete this document:

In [1]:
import sys
import os
import pickle
import numpy as np

testPath = '/'.join(os.getcwd().split('/')[:-1]) + "/Tester"
sys.path.insert(0, testPath )
pickleFile = testPath+ "/SparkBasics1.pkl"

import Tester
import SparkBasics1

### Spark Context
* Spark is complex distributed doftware. 
* The python interface to spark is called **pyspark**
* **SparkContext** is a python class, defined as part of **pyspark** which manages the communication between the user's program and spark.

We start by creating a **SparkContext** object named **sc**. In this case we create a spark context that uses 3 *executors*

In [2]:
from pyspark import SparkContext
sc = SparkContext(master="local[4]")
sc

### Only one sparkContext at a time!
When you run spark in local mode, you can have only a single context at a time. Therefor, if you want to use spark in a second notebook, you should first stop the one you are using here. This is what the method `.stop()` is for.

In [3]:
# sc.stop() #commented out so that you don't stop your context by mistake

<h3>RDDs</h3>

<p>RDD (or Resilient Distributed DataSet) is the main novel data structure in Spark. You can think of it as a list whose elements are stored on several computers.</p>

<p><img alt="" src="Figures/SparkContextAndRDD.jpg" style="height:324px; width:900px" /></p>


The elements of each `RDD` are distributed across the **worker nodes** which are the nodes that perform the actual computations. This notebook, however, is running on the **Driver node**. As the RDD is not stored on the driver-node you cannot access it directly. The variable name `RDD` is really just a pointer to a python object which holds the information regardnig the actual location of the elements.

#### Parallelize 
* Simplest way to create an RDD.
* The method `A=sc.parallelize(L)`, creates an RDD named `A` from list `L`.
* `A` is an RDD of type `ParallelCollectionRDD`.

In [3]:
A=sc.parallelize(range(3))
A

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:475

#### Collect

* RDD content is distributed among all executors.
* `collect()` is the inverse of `parallelize()'
* collects the elements of the RDD
* Returns a list


In [4]:
L=A.collect()
print type(L)
print L

<type 'list'>
[0, 1, 2]


### Using `.collect()` eliminates the benefits of parallelism
It is often tempting to `.collect()` and RDD, make it into a list, and then process the list using standard python. However, note that this means that you are using only the head node to perform the computation which means that you are not getting any benefit from spark.

Using RDD operations, as described below, **will** make use of all of the computers at your disposal.

### Map
* applies a given operation to each element of an RDD
* parameter is the function defining the operation.
* returns a new RDD.
* Operation performed in parallel on all executors.
* Each executor operates on the data **local** to it.

**Note:** Here we are using **lambda** functions, later we will see that regular functions can also be used.

For more on lambda function see [here](http://www.secnetix.de/olli/Python/lambda_functions.hawk)

#### Excercise 1

1. Write a function called `mapcos` that has a single paramater: an RDD of numbers, computes the `cos()` (cosine) and returns it as a list.

```cos(A)``` should produce the output
    
```
    [1.0, 0.54030..., -0.41614...]
```

**Approximate answers** The reason that the numbers above end with `...` is that, when computing with floating point numbers, there are always roundoff errors, and the roundoff errors may depend on the architecture of the computer you are using. To avoid this problem, we test the answers you produce within a tolerance of 0.1%, in other words if your answer is $a$ and the correct answer is $b$ then your answer is considered correct if
$$ \left| \frac{a-b}{a} \right| < 0.001$$

### Exercise 1: Teacher Stuff

In [3]:
import SparkBasics1_Teacher

In [8]:
def mapcos(A):
    return A.map(np.cos)

In [11]:
inputs= [ sc.parallelize(range(3)),
          sc.parallelize(range(4,12)), 
          sc.parallelize(range(-4,0))   ]

Tester.GenPickle(mapcos, inputs, pickleFile, "ex1" )

In [9]:
# after creating the code in the SparkBasics1_Teacher, we check that it works belows
import SparkBasics1_Teacher
SparkBasics1_Teacher.exercise1(testPath, mapcos, sc)

Input: [3.141592653589793, 0, 100000, 31.41592653589793, 3.784189633918261, -49.4902]
Correct Output: [-1.0, 1.0, -0.99936080743821243, 1.0, -0.80054215002569207, 0.71422336555046395]
Great Job!

Input: [39, 40, 41, 42, 43, 44]
Correct Output: [0.26664293235993725, -0.66693806165226188, -0.98733927752382644, -0.39998531498835127, 0.55511330152062566, 0.99984330864769122]
Great Job!



### Exercise 1:

In [4]:
def mapcos(A):
    # Write the code that will perform the task outlined above
    return "return spark FUNCTION here, so to get a list run: mapcos(A).collect() "

In [13]:
SparkBasics1.exercise1(pickleFile, mapcos,sc)

Input: [0, 1, 2]
Correct Output: [1.0, 0.54030230586813977, -0.41614683654714241]
Great Job!

Input: [4, 5, 6, 7, 8, 9, 10, 11]
Correct Output: [-0.65364362086361194, 0.28366218546322625, 0.96017028665036597, 0.7539022543433046, -0.14550003380861354, -0.91113026188467694, -0.83907152907645244, 0.0044256979880507854]
Great Job!

Input: [-4, -3, -2, -1]
Correct Output: [-0.65364362086361194, -0.98999249660044542, -0.41614683654714241, 0.54030230586813977]
Great Job!



### Excercise 2
Consider the following RDD: 

```
stringRDD=sc.parallelize(["Spring quarter", "Learning spark basics", "Big data analytics with Spark"])
```
   Write a function called `mapwords` that has a single paramater: an RDD of strings, and returns a list of words for each string.  
    ```mapwords(MapString)``` should produce the output:
    
``` 
[['Spring', 'quarter'], ['Learning', 'spark', 'basics'], ['Big', 'data', 'analytics', 'with', 'Spark']]
```

# Exercise 2:  Teacher Stuff

In [9]:
def mapwords(stringRDD):
    return stringRDD.map(lambda x: x.split() )

In [5]:
inputs = [ sc.parallelize(["Spring quarter", "Learning spark basics", "Big data analytics with Spark"]),
           sc.parallelize(["Do not go gentle", "into that good night", "old age should burn and rave"]),
           sc.parallelize(["do","I dare disturb","the universe","there will be time there will be","time"]) ]

Tester.GenPickle(mapwords, inputs, pickleFile, "ex2" )

In [6]:
# after creating the code in the SparkBasics1_Teacher, we check that it works belows
SparkBasics1_Teacher.exercise2(testPath, mapwords, sc)

Input: ['ah', 'ah ah ah', 'ha ai ifo aoisdmf']
Correct Output: [['ah'], ['ah', 'ah', 'ah'], ['ha', 'ai', 'ifo', 'aoisdmf']]
Great Job!

Input: ['asdio', 'i', 'asdfasd', 'aasdf']
Correct Output: [['asdio'], ['i'], ['asdfasd'], ['aasdf']]
Great Job!

Input: ['do asdnj aksdo adsof aos asod oasdf  mkmasdkf maso asdf okm']
Correct Output: [['do', 'asdnj', 'aksdo', 'adsof', 'aos', 'asod', 'oasdf', 'mkmasdkf', 'maso', 'asdf', 'okm']]
Great Job!



# Exercise 2:

In [7]:
def mapwords(B):
    # Write the code that will perform the task outlined in exercise 2
    return "return spark FUNCTION here, so to get a list run: mapwords(B).collect() "

In [10]:
SparkBasics1.exercise2(pickleFile, mapwords, sc)

Input: ['Spring quarter', 'Learning spark basics', 'Big data analytics with Spark']
Correct Output: [['Spring', 'quarter'], ['Learning', 'spark', 'basics'], ['Big', 'data', 'analytics', 'with', 'Spark']]
Great Job!

Input: ['Do not go gentle', 'into that good night', 'old age should burn and rave']
Correct Output: [['Do', 'not', 'go', 'gentle'], ['into', 'that', 'good', 'night'], ['old', 'age', 'should', 'burn', 'and', 'rave']]
Great Job!

Input: ['do', 'I dare disturb', 'the universe', 'there will be time there will be', 'time']
Correct Output: [['do'], ['I', 'dare', 'disturb'], ['the', 'universe'], ['there', 'will', 'be', 'time', 'there', 'will', 'be'], ['time']]
Great Job!



### Reduce

* Takes RDD as input, returns a single value.
* **Reduce operator** takes **two** elements as input returns **one** as output.
* Repeatedly applies a **reduce operator**
* Each executor reduces the data local to it.
* The results from all executors are combined.

The simplest example of a 2-to-1 operation is the sum:

In [17]:
A.reduce(lambda x,y: x+y)

3

Here is an example of a reduce operation that finds the shortest string in an RDD of strings.

In [18]:
words=['this','is','the','best','mac','ever']
wordRDD=sc.parallelize(words)
wordRDD.reduce(lambda w,v: w if len(w)<len(v) else v)

'is'

#### Exercise 3

1. Write a `reduce` command that outputs the maximum number from a list of numbers. Your command should produce the following output on the input ```RDD=sc.parallelize([0,2,1])```

   Output: ``` 2 ```

# Exercise 3: Teacher Stuff

In [19]:
def getMax(C):
    return C.reduce(max)

In [20]:
inputs = [ sc.parallelize([0,4,2,3,1]),
           sc.parallelize([-3.2,-3.233,-3.1,-3.9]),
           sc.parallelize([2,2,2,2,2,2]) ]

Tester.GenPickle(getMax, inputs, pickleFile, "ex3", isRDD=False )

In [21]:
# after creating the code in the SparkBasics1_Teacher, we check that it works belows
SparkBasics1_Teacher.exercise3(testPath, getMax, sc)

Input: [0, 3.49, 2.4922, 5.24, -24]
Correct Output: 5.24
Great Job!

Input: [-3.01, -3.2, -3.001, -3.4]
Correct Output: -3.001
Great Job!

Input: [7]
Correct Output: 7
Great Job!



# Exercise 3

In [ ]:
def getMax(C):
    # Write the code that will perform the task outlined in exercise 3
    return "return spark NUMBER here, example, to get a number run mapwords(C)"

In [22]:
SparkBasics1.exercise3(pickleFile, getMax, sc)

Input: [0, 4, 2, 3, 1]
Correct Output: 4
Great Job!

Input: [-3.2, -3.233, -3.1, -3.9]
Correct Output: -3.1
Great Job!

Input: [2, 2, 2, 2, 2, 2]
Correct Output: 2
Great Job!



### Exercise 4

2. Consider the stringRDD defined in Exercise. Write a `reduce` command to produce a single string which is the concatenation of all the strings in stringRDD(with a space between each string). You output should look like:

    Output: ``` 'Spring quarter Learning spark basics Big data analytics with Spark' ```

# Teacher Stuff

# Is this one correct?

In [21]:
for i in inputs:
    print mapwords2(mapwords(i))

['Spring', 'quarter', 'Learning', 'spark', 'basics', 'Big', 'data', 'analytics', 'with', 'Spark']
['Do', 'not', 'go', 'gentle', 'into', 'that', 'good', 'night', 'old', 'age', 'should', 'rave', 'and', 'burn']
['do', 'I', 'dare', 'disturb', 'the', 'universe', 'there', 'will', 'be', 'time', 'there', 'will', 'be', 'time']


In [11]:
def mapwords2(mapwords):
    return mapwords.reduce(lambda x,y: x+" "+y)

In [12]:
inputs = [ sc.parallelize(["Spring quarter", "Learning spark basics", "Big data analytics with Spark"]),
           sc.parallelize(["Do not go gentle", "into that good night", "old age should burn and rave"]),
           sc.parallelize(["do","I dare disturb","the universe","there will be time there will be","time"]) ]

Tester.GenPickle(mapwords2, inputs, pickleFile, "ex4", isRDD=False )

In [13]:
SparkBasics1_Teacher.exercise4(testPath, mapwords2, sc)

TypeError: can only concatenate list (not "str") to list

# Exercise 4

In [27]:
def mapwords2(mapwords):
    # This function should take in as it's input the previous mapwords function you wrote
    # you should then apply 1 spark function to it to it so that you return a single list
    return "return something like: mapwords.somefunction(...)"

In [14]:
SparkBasics1.exercise4(pickleFile, mapwords2, sc)

Input: ['Spring quarter', 'Learning spark basics', 'Big data analytics with Spark']
Correct Output: Spring quarter Learning spark basics Big data analytics with Spark
Great Job!

Input: ['Do not go gentle', 'into that good night', 'old age should burn and rave']
Correct Output: Do not go gentle into that good night old age should burn and rave
Great Job!

Input: ['do', 'I dare disturb', 'the universe', 'there will be time there will be', 'time']
Correct Output: do I dare disturb the universe there will be time there will be time
Great Job!



### Using regular functions instead of lambda functions

* lambda function are short and sweet.
* but sometimes it's hard to use just one line.
* We can use full-fledged functions instead.

suppose we want to find the 
* last word in a lexicographical order 
* among 
* the longest words in the list.

We could achieve that as follows

In [ ]:
def largerThan(x,y):
    if len(x)>len(y): return x
    elif len(y)>len(x): return y
    else:  #lengths are equal, compare lexicographically
        if x>y: 
            return x
        else: 
            return y
        
wordRDD.reduce(largerThan)

#### Exercise 5

1. Consider the following RDD:

    ``` listRDD=sc.parallelize([[3,4],[2,1],[7,9]]) ```
 
     Write a regular function with `reduce` command to output the maximum element from a set of lists. Your output should look like:
     
     Output: ```[9]```
     
     (Note: The output is a list containing a single number rather than just a single number)

# Teacher Stuff

In [ ]:
listRDD=sc.parallelize([[3,4],[2,1],[7,9]])

In [ ]:
getMax2(listRDD)

In [ ]:
def getMax2(A): 
    return A.reduce(lambda x,y: [max(x+y)])

In [ ]:
inputs = [ sc.parallelize([[3,4],[2,1],[7,9]]),
           sc.parallelize([[-222],[-10,-33],[0,-5]]),
           sc.parallelize([[3.2,3.3,3.1,3.9],[-3.95],[3.4,3.7]]) ]

Tester.GenPickle(mapwords2, inputs, pickleFile, "ex5", isRDD=False )

In [ ]:
SparkBasics1_Teacher.exercise5(testPath, getMax2, sc)

# Exercise 5

In [ ]:
def getMax2(A):
    # doe something
    return "write something"

In [ ]:
SparkBasics1.exercise5(pickleFile, getMax2, sc)

<h3>Reduce operations <strong>must not depend on the order</strong></h3>

<ul>
	<li>Order of operands should not matter</li>
	<li>Order of application of reduce operator should not matter</li>
</ul>

<p>Multiplication and summation are good:</p>

<h1>&nbsp; &nbsp; &nbsp; &nbsp; 1 + 3 + 5 + 2 &nbsp; &nbsp; &nbsp;5 + 3 + 1 + 2 &nbsp;</h1>


<p>Division and subtraction are bad:</p>

<h1>&nbsp; &nbsp; &nbsp; &nbsp; 1 - 3 - 5 - 2 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;1 - 3 - 5 - 2 &nbsp;</h1>


### Why must reordering not change the result?

You can think about the reduce operation as a binary tree where the leaves are the elements of the list and the root is the final result. Each triplet of the form (parent, child1, child2) corresponds to a single application of the reduce function. 

The order in which the reduce operation is applied is **determined at run time** and depends on how the RDD is partitioned across the cluster.
There are many different orders to apply the reduce operation. 

If we want the input RDD to uniquely determine the reduced value **all evaluation orders must must yield the same final result**. In addition, the order of the elements in the list must not change the result. In particular, reversing the order of the operands in a reduce function must not change the outcome. 

For example the arithmetic operations multiply `*` and add `+` can be used in a reduce, but the operations subtract `-` and divide `/` should not.

Doing so will not raise an error, but the result is unpredictable.

Here is a example  
Which of these the following orders was executed?
* $$((1-3)-5)-2$$ or
* $$(1-3)-(5-2)$$

In [ ]:
B=sc.parallelize([1,3,5,2])
B.reduce(lambda x,y: x-y)